In [4]:
#!/usr/bin/env python3
"""
fetch_qqq_calls_sorted.py

Fetch all end‑of‑day CALL option data for QQQ from EODHD,
ordering expirations ascending so you get nearest‐term contracts first,
and write all contract attributes to CSV sorted by DTE ascending.
"""

import requests
import pandas as pd
import time
import sys
from datetime import datetime, timedelta

# ─── CONFIGURATION ─────────────────────────────────────────────────────────────
API_TOKEN   = "67fb3d6f50c489.92544905"  # Your EODHD API token
SYMBOL      = "QQQ"
END_DATE    = datetime.today().strftime("%Y-%m-%d")
START_DATE  = (datetime.today() - timedelta(days=365)).strftime("%Y-%m-%d")
BASE_URL    = "https://eodhd.com/api/mp/unicornbay/options/eod"
PAGE_LIMIT  = 1000                        # Per EODHD docs
PAUSE_SEC   = 0.6                         # To respect rate limits (1000 req/min)

# ─── FETCH ALL CALL OPTIONS WITH ASCENDING EXPIRY ──────────────────────────────
def fetch_all_calls():
    all_records = []
    offset = 0

    while True:
        params = {
            "api_token":                 API_TOKEN,
            "filter[underlying_symbol]": SYMBOL,
            "filter[tradetime_from]":    START_DATE,
            "filter[tradetime_to]":      END_DATE,
            "filter[type]":              "call",
            "page[limit]":               PAGE_LIMIT,
            "page[offset]":              offset,
            "sort":                      "exp_date",  # ascending by expiration date
            "compact":                   "0",
        }
        resp = requests.get(BASE_URL, params=params)
        # stop on 422/404 to avoid errors at high offsets
        if resp.status_code in (422, 404):
            break
        resp.raise_for_status()

        data = resp.json().get("data", [])
        if not data:
            break

        all_records.extend(data)
        print(f"Fetched {len(data)} rows @ offset {offset} (total={len(all_records)})")
        if len(data) < PAGE_LIMIT:
            break

        offset += PAGE_LIMIT
        time.sleep(PAUSE_SEC)

    return all_records

# ─── MAIN ──────────────────────────────────────────────────────────────────────
def main():
    print(f"🔍 Fetching all CALL data for {SYMBOL} ({START_DATE}→{END_DATE})…")
    records = fetch_all_calls()
    if not records:
        print("❌ No records fetched — check API token or date range.", file=sys.stderr)
        sys.exit(1)

    # Flatten JSON into DataFrame
    df = pd.json_normalize(records)

    # Convert DTE to numeric and sort ascending
    df["dte"] = pd.to_numeric(df["attributes.dte"], errors="coerce")
    df_sorted = df.sort_values("dte")

    # Write to CSV
    output_file = f"{SYMBOL}_calls_sorted_by_dte.csv"
    df_sorted.to_csv(output_file, index=False)
    print(f"✅ Wrote {len(df_sorted)} records to {output_file}")

if __name__ == "__main__":
    main()



🔍 Fetching all CALL data for QQQ (2024-04-16→2025-04-16)…
Fetched 1000 rows @ offset 0 (total=1000)
Fetched 1000 rows @ offset 1000 (total=2000)
Fetched 1000 rows @ offset 2000 (total=3000)
Fetched 1000 rows @ offset 3000 (total=4000)
Fetched 1000 rows @ offset 4000 (total=5000)
Fetched 1000 rows @ offset 5000 (total=6000)
Fetched 1000 rows @ offset 6000 (total=7000)
Fetched 1000 rows @ offset 7000 (total=8000)
Fetched 1000 rows @ offset 8000 (total=9000)
Fetched 1000 rows @ offset 9000 (total=10000)
Fetched 1000 rows @ offset 10000 (total=11000)
✅ Wrote 11000 records to QQQ_calls_sorted_by_dte.csv
